In [1]:
import pandas as pd
import requests
from requests import request
from bs4 import BeautifulSoup
from multiprocessing.dummy import Pool as ThreadPool
import functools
import time
from tqdm.notebook import tqdm
import sys
from itertools import chain
import re
import functools
import pymorphy2

In [10]:
import pandas as pd 
import datetime
import numpy as np 
import matplotlib.pyplot as plt 

In [17]:
import pandas as pd
from bs4 import BeautifulSoup
import bs4
from multiprocessing import Pool, Lock, Value
from time import sleep
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
import base64
import numpy as np
import zlib
from itertools import product
import re

In [2]:
import pymorphy2  # $ pip install pymorphy2
  
def pos(word, morth=pymorphy2.MorphAnalyzer()):
    "Return a likely part of speech for the *word*."""
    return morth.parse(word)[0].tag.POS
  
words = "Однако я так и не смог закончить".split()
functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'}  # function words
# p2 = [word for word in p1 if (pos(word) not in functors_pos and len(word)>1)]
# -> я смог закончить

In [3]:
list_mest = ['я', 'ты', 'он', 'она', 'оно', 'мы',
             'вы', 'они','мой', 'твой', 'свой', 'ваш', 'наш', 'его', 'её', 'их',
            'все','быть','может', 'можно', 'очень','нужно','будет','уже','могут',
            'кто', 'что', 'какой', 'чей', 'где', 'который', 'откуда', 'сколько', 'каковой', 'каков', 'зачем',
            'меня','мне','того', 'этого', 'этот','ее','вы','более', 'было', 'вам', 'все', 'вы',  
             'которые', 'нет',  'том', 'этом',
            'вас', 'вообще', 'всего', 'вы',
            'себе', 'себя', 'ему', 'тебе', 'тебя','был']

In [4]:
morph = pymorphy2.MorphAnalyzer()

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [6]:


def batch_generator(X, y, shuffle=True, batch_size=1):
    """
    Гератор новых батчей для обучения
    X          - матрица объекты-признаки
    y_batch    - вектор ответов
    shuffle    - нужно ли случайно перемешивать выборку
    batch_size - размер батча ( 1 это SGD, > 1 mini-batch GD)
    Генерирует подвыборку для итерации спуска (X_batch, y_batch)
    """
    ind = np.array(range(y.shape[0]), dtype = int)
    if(shuffle):
        ind = np.random.permutation(y.shape[0])
    for i in range(len(y)//batch_size):
        X_batch = X[ind[i*batch_size:(i+1)*batch_size],:]
        y_batch = y[ind[i*batch_size:(i+1)*batch_size]]
        yield (X_batch, y_batch)

In [7]:
from sklearn.base import BaseEstimator, ClassifierMixin


def sigmoid(x):
    """
    Вычисляем значение сигмоида.
    X - выход линейной модели
    """
    # Your code Here
    sigm_value_x = 1/(1 + np.exp(-(x)))
    return sigm_value_x


class MySGDClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, batch_generator, C=1, alpha=0.01,
                 max_epoch=10, model_type='lin_reg', batch_size=1, th=0.5):
        """
        batch_generator -- функция генератор, которой будем создавать батчи
        C - коэф. регуляризации
        alpha - скорость спуска
        max_epoch - максимальное количество эпох
        model_type - тим модели, lin_reg или log_reg
        """
        self.th = th
        self.C = C
        self.alpha = alpha
        self.max_epoch = max_epoch
        self.batch_generator = batch_generator
        self.errors_log = {'iter' : [], 'loss' : []}  
        self.model_type = model_type
        self.batch_size = batch_size

    def calc_loss(self, X_batch, y_batch):
        """
        Считаем функцию потерь по батчу 
        X_batch - матрица объекты-признаки по батчу
        y_batch - вектор ответов по батчу
        Не забудте тип модели (линейная или логистическая регрессия)!
        """
        if (self.model_type == 'lin_reg'):
            loss = 1 / y_batch.shape[0] * ((np.dot(X_batch, self.weights.reshape(-1, 1)) - y_batch) ** 2).sum() 
            + 1 / self.C * ((self.weights * self.weights).sum())
            return loss
        else:
            sigm = np.arange(y_batch.shape[0], dtype = float)
            for i in range(y_batch.shape[0]):
                sigm[i] = sigmoid((X_batch[i, : ] * self.weights).sum())
            loss = - (y_batch * np.log(sigm) + (1 - y_batch) * np.log(1 - sigm)).sum() / y_batch.shape[0] 
            + 1 / self.C * ((self.weights * self.weights).sum())
            return loss

    def calc_loss_grad(self, X_batch, y_batch):
        """
        Считаем  градиент функции потерь по батчу (то что Вы вывели в задании 1)
        X_batch - матрица объекты-признаки по батчу
        y_batch - вектор ответов по батчу
        Не забудте тип модели (линейная или логистическая регрессия)!
        """
        if (self.model_type == 'lin_reg'):
            loss_grad = (2/y_batch.shape[0])*np.dot((np.dot(X_batch, self.weights.reshape(-1,1)) 
                                        - y_batch.reshape(-1,1)).T,X_batch).sum(axis = 0) + 2/self.C*self.weights
            return loss_grad.reshape(-1)
        else:
            sigm = np.arange(y_batch.shape[0], dtype = float)
            for i in range(y_batch.shape[0]):
                sigm[i] = sigmoid((X_batch[i, : ]*self.weights).sum())
            loss_grad = np.dot(X_batch.T,(sigm - y_batch).reshape(-1,1)).reshape(self.weights.shape[0])
            + 2/self.C*self.weights
            return loss_grad

    def update_weights(self, new_grad):
        """
        Обновляем вектор весов
        new_grad - градиент по батчу
        """
        self.weights -= self.alpha *new_grad

    def fit(self, X, y):
        '''
        Обучение модели
        X - матрица объекты-признаки
        y - вектор ответов
        '''
        # Нужно инициализровать случайно веса
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        self.weights = np.random.random(X.shape[1])
        for n in range(0, self.max_epoch):
            new_epoch_generator = self.batch_generator(X, y, batch_size = self.batch_size)
            for batch_num, new_batch in enumerate(new_epoch_generator):
                X_batch = new_batch[0]
                y_batch = new_batch[1]
                batch_loss = self.calc_loss(X_batch, y_batch)
                batch_grad = self.calc_loss_grad(X_batch, y_batch)
                self.update_weights(batch_grad)
                # Подумайте в каком месте стоит посчитать ошибку для отладки модели
                # До градиентного шага или после
                self.errors_log['iter'].append(batch_num)
                self.errors_log['loss'].append(batch_loss)
        return self

    def predict(self, X):
        '''
        Предсказание класса
        X - матрица объекты-признаки
        Не забудте тип модели (линейная или логистическая регрессия)!
        '''
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        if (self.model_type == 'lin_reg'):
            y_hat = np.dot(X, self.weights)/np.sum(X)
        else:
            sigm = [[] for i in range(X.shape[0])]
            for i in range(X.shape[0]):
                sigm[i] = sigmoid((X[i, : ]*self.weights).sum())
            y_hat = (np.array(sigm))
        # Желательно здесь использовать матричные операции между X и весами, например, numpy.dot 
        return y_hat

In [55]:
import re
from pymystem3 import Mystem
doc_to_t = {}
m = Mystem()
with open('docs_titles.tsv') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
#         cur = re.split(r' ',title)
        r = [ i.lower() for i in re.split(r'\W+',title) if i]
        p = [re.sub(r'[^А-я]', '', i) for i in r]
        p1 = list(filter(lambda i: i, p))
        cur = [word for word in p1 if (pos(word) not in functors_pos and len(word)>1)]
        cur = [u.lower() for u in cur if not u.lower() in list_mest]
        cur = m.lemmatize(' '.join(cur))
#         cur = [morph.parse(o)[0].normal_form for o in cur]
        
        title = ''
        for i in cur:
            if len(i)>2:
                title += i + ' '
        print(title)
        doc_to_t[doc_id] = title
print (len(doc_to_t))

ваз замена подшипник ступица нива 
ваз опт сочи сравнивать цена купить потребительский товар 
купить ступица лад калина трансмиссия переходный ступица цена замена тюнинг 
классика 
ступица нива замена подшипник свой рука 
ваз 
обзор подшипник полуось ваз 
купить подшипник ступица страница 
автомобильный интернет портал отзыв владелец ваз нива год 
новость сообщение официальный группа вконтакте торговый компания магазин тольятти 
инструкция замена подшипник передняя ступица ивеко дейли 
ступица страница 
маааленькая проблемка бортжурнал авток год 
разгружать полуось нива шлиц 
прошивка нива скачать файлообменник 
страница раздел каталог подвеска 
продавать нива 
рекомендация проведение независимый экспертиза осаго 
втулка подшипник омск сравнивать цена поставщик промышленный товар 
шевроле блейзер елабуга руководство эксплуатация инструкция совет 
привод шрус пыльник шрусый страница 
автомобиль маз калининград интернет магазин компания стоимость каталог купить автомобиль маз цена сайт 


история законодательство право поземельный собственность россия начало век 
печать страница сьезд вкп сьезд расстреливать победитель 
административный деятельность ордин нащокина сфера местный управление 
помогать слива похудеть 
часть первый несбывшийся надежда агония кровавый дорога отступление 
психология экстремальный ситуация 
аншин центр репродукция генетика фертимед москва 
комплексный нормативный словарь современный русский язык 
диплом гражданский право процесс процедура заключение международный договор 
день цикл сдавать анализ гормон ттг женщина 
энтони саттон кулиса русский революция уолл стрит большевистский революция конспирология 
читать онлайн большой советский энциклопедия большой советский энциклопедия ока электронный библиотека 
анализ гормон женщина климакс женский совет 
символист другой статья разыскание публикация 
форум теплопункт 
гость екатеринбург онлайн 
оружейный новость архив страница 
день цикл сдавать ттг 
вопрос ответ тема овуляция ановуляция страница к

какой команда самп викидывать машын какой команда починка машина самп какой команда ставить шип самп какой команда самп облакотиться ставить шип самп установка продолжать установка ключ установка продолжение установка вводить верификация установка хвоха установка активация смс вводить установка искать банда банда гта онлайн допустимый титул банда гта онлайн исследовательский банда гта онлайн 
код нанимать лбово сан андреас код огненный тачка сан андреас код сан андреас стрибок код гытыа сан андреас мотоцикл код женица сан андреас код одяга джит сан андреас код гатыа сан андреас полицейский гоа сан андреас код коди супер героить сан андреас код магнит шлах сан андреас скачать чита гта сан андреас миниган скачать чита гта сан андреас апакалиса скачать чит телепорт гта сан андреас скачать чит джетпак гта сан андреас андроид скачать чит бакс гта сан андреас мультиплеер 
видео гта прикол алекс брейн видео гта алекс брейный танк смотреть гта алекс брейный танк смотреть гта алекс брейн бикман

штора кухня 
ответ подсказывать длина должный штора лежать пол пол 
длина должный штора правильный проведение расчет 
длина должный штора гостиная спальня пример фото штора гостиная спальня 
интернет магазин салон штора ткань хельга статья длина штора подбирать штора длина 
подбирать длина штора какой должный длина штора блиц текстиль 
длина должный штора текстиль лен штора тюль ткань ооо нана трейдинг уход штора 
длина должный штора 
новогодний стол новогодний рецепт рецепт новый год фото 
расчет количество ткань штора текстиль дом штора тюль пошив заказ 
сшивать штора свой рука 
штора форум хороший подруга 
правильно рассчитать длина карниз штора 
правильно подбирать штора дом квартира среда обитание 
персональный сайт полезный совет 
длина штора зависеть высота комната длина должный штора 
какой должный длина штора фото штора гостиная шить штора 
сморщенный почка нефросклероз почка причина симптом лечение 
пьм вода 
холестерин кровь содержание норма повышать метод снижение 
триместр

эффективный народный лечение туберкулез народный средство прополис медведка 
лечиться туберкулез 
долго лечиться туберкулез консультация раздел туберкулез 
туберкулез болезнь бедный требовать денежный лечение 
лечить туберкулез народный средство 
долго лечиться туберкулез легкое 
вопрос время лечиться туберкулез ребенок 
долго лечиться туберкулез форум 
туберкулз вылечиваться форум 
лечение туберкулез симптом признак туберкулез 
настройка создание форум 
просмотр тема новость 
секрет новость мир игра виртуальный реальность 
сделать маленький прицел 
устанавливать конфиг 
новость 
сделать маленький прицел настройка прицел 
военный литература мемуары ситковский небо сокол 
сделать сразу покупаласить грена 
страница форум халява 
мода прицел фото открывать капсула 
новость 
смена прицел форум халява 
код гта иксбокс 
рассказ паутина черный вдова 
дневник радужный мост российский сервис онлайн дневник 
точность прицел 
шутер андроид русский 
михаил блажчук вконтакте 
дэн симмонс эндимион 


...

сертификация стандартизация плодовоощной продукция 
компот абрикос слива зима рецепт кулинария 
материнство меню малыш день мес 
варение компот джем другой сладость форум 
замораживать слива грн продукт питание напиток харьков 
вишневый компот зима рецепт 
замораживать ягода зимний вкус лето мир глаз мама 
компот свежий абрикос слива компот пошаговый рецепт фото 
полезный совет закон здоровье 
похудение королек 
алыча википедия 
курсовой изысканный салат 
ященко забота радость фермер 
заголовок обсуждение российский сервис онлайн дневник 
заголовок обсуждение российский сервис онлайн дневник 
рецепт компот слива абрикос 
логопедический занятие старший группа смола логопед 
диета поправляться диета портал узбекистан 
сделать компот абрикос слива 
неизвестный автор книга вкусный здоровый пища 
материнство вкуснятина творог 
заготовка фрукт запись рубрика заготовка фрукт дневник верабарышев российский сервис онлайн дневник 
приготовление фрукт домашний настойка 
абрикос 
саженец дерево ку

хламидия 
курсовой работа разбегание галактика роль эволюция вселять 
вконтакте 
реферат квантовый пастулат бор 
полировка кузов нанесение защитный покрытие бортжурнал чемпион год 
уход кузов кузов 
способ регистрация заряжать частица радько устройство осуществление вариант патент радько валерий евгеньевич 
полироль фольксваген гольф клуб 
герпес назолабиальный 
конец новый свет крах сша страница знак исполнение времна 
уход лакокрасочный покрытие следопыт 
предисловие океанский простор 
галактика структура вселенная 
приходить посылочка японский химия бортжурнал год 
пользоваться жидкий стекло вилсон инструкция применение вид полироль 
полироль жидкий стекло инструкция нанесение 
купить жидкий стекло рубцовск предложение покрытие жидкий стекло вилсон цена 
мука метод микробиологический контроль готовый изделие крем 
курсовой работа эволюция вселенная технический библиотека 
страница файл книга 
правильно покрывать автомобиль жидкий стекло 
пнина 
презентация тема безопасность жизнедея

джеки фильм онлайн фильмография 
джеки чан чжа хороший книга 
час пик смотреть онлайн бесплатно 
тег джеки чан новость днепропетровск сегодня событие днепр последний новость днепропетровск область украина днепр вечерний 
кошмар джеки чан яплакать 
джеки чан пропадать семья терабит новый фильм сериал игра саундтрек 
карате пацан скачать фильм бесплатно мобильный телефон 
джеки чан фильм онлайн 
джеки чан получать подчетный оскар церемония 
фильм карате пацан смотреть онлайн 
история знакомство джеки чан начинаться страна мама 
операция феникс смотреть онлайн бесплатно 
час пик смотреть онлайн бесплатно 
читать онлайн книга семь толстый ткач павел блинников 
джеки чан привозить москва доспехи бог спеть поклонник новость общество новость 
история успех джеки чан 
фотография близнец фильм джеки чан 
поиск джеки кино гонконг китай тайвань 
новый игрушка джеки чан личный самолет млн доллар яплакать 
рецензия фильм великолепный кино гонконг китай тайвань 
сериал приключение джеки чан видео 
ф

хороший программа блокировка реклама всплывать окно скачать бесплатно 
защита реклама интернет убирать реклама помощь программа 
убирать реклама интернет отключать всплывать окно 
скачать бесплатно программа блокировка реклама браузер опера яндекс браузер хром мазила 
интернет анти спам скачать программа 
защищать домашний компьютер вирус шпионский программа компьютер дом сборка настройка обслуживание ремонт 
избавляться реклама браузер 
защита реклама браузер 
удалять вирус троян всплывать окно попапа шпионский программа прочий гадость компьютер скачать хороший бесплатный антивирус 
опыт блокировка рекламный окно помощь 
программа блокировка всплывать окно любой другой вид реклама 
реклама браузер убирать всплывать окно реклама браузер ремонт компьютер троещин дом компьютерный помощь диагностика компьютер троещина 
блокировать реклама 
убирать реклама браузер навсегда удалять всплывать окно весь сайт 
почистить браузер вирус реклама программа удаление реклама браузер компьютер 
вирусн

видео книга пособие автомаляр инструкция совет мануал 
двигатель тойота вист форум 
установка контроллер давление масло 
запчасть казахстан сравнивать цена купить потребительский товар 
автовопрос архив страница рыбалка образ жизнь 
класс ошибка форум автодиагностика автосканер ремонт обслуживание эксплуатация автомобиль 
двигатель коробка передача усть каменогорск сравнивать цена купить потребительский товар 
маленький трагедия 
стена вконтакте 
первый очередь заблуждаться альфаподбор подбор авто спб покупка авто вконтакте 
мерседес спринтер отзыв владелец авто центр отзыв страница 
класс выбирать оптимальный стиль чоппер 
руководство ремонт руководство эксплуатация 
отзыв автовладелец 
привозить двигатель белоруссия архив форум 
инструкция эксплуатация 
помогать узнавать тип двигатель мерседес клуб форум мерседес 
двигатель техника нижний новгород сравнивать цена купить потребительский товар стр 
движок поставлять год форум 
дансинг ставка гитлер либрусек 
получать новый документ офо

In [56]:
import pandas as pd
train_d = pd.read_csv('train_groups.csv')
traingroups_titled = {}
for i in range(len(train_d)):
    new_doc = train_d.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_t[doc_id]
#     title = [u.lower() for u in title if not u.lower() in list_mest]
#     title = [morph.parse(o)[0].normal_form for o in title]
    if doc_group not in traingroups_titled:
        traingroups_titled[doc_group] = []
    traingroups_titled[doc_group].append((doc_id, title, target))

In [ ]:
# %%time
# # вот этот кусок кода!!!!!!!!!!!!!!!!!!!

# import numpy as np
# import Levenshtein
# import numpy as np
# import gensim
# import nltk as nl
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# from gensim.models import Word2Vec
# y_train1 = []
# X_train1 = []
# groups_train1 = []
# a = 0
# b = 1000
# for new_group1 in traingroups_titled:
#     docs1 = traingroups_titled[new_group1]
#     list_data = []
#     ls = []
#     aa = 0
#     for k, (doc_id, title, target_id) in enumerate(docs1):
#         aa+=1
#         title = title.strip().split()
#         title1 = [u.lower() for u in title if not u.lower() in list_mest]
#         if len(title1)==0:
#             for e in range(10):
#                 title1.append(str(k))
#         elif len(title1)<10:
#             for e in range(10-len(title1)):
#                 title1.append(title1[0])
#         ls.append(title1[:10])
#         list_data.append(' '.join(title1))
#     print(aa)
#     print(len(ls))
#     vectorizer = TfidfVectorizer(max_features=20)
#     model = Word2Vec(ls,size = 10)
#     wordss = list(model.wv.vocab)
#     print(wordss)
#     vectorss = []
#     for word in wordss:
#         vectorss.append(model[word].tolist())
#     data11 = np.array(vectorss)
#     data11 = np.sort(data11, axis=0)
#     print(data11.shape)
#     vectorss = []
#     for word in wordss:
#         vectorss.append(model[word].tolist())
#     X6 = vectorizer.fit_transform(list_data)
#     dic = set(vectorizer.get_feature_names())
#     print(dic)
#     vectorizer = CountVectorizer(max_features=20).fit_transform(list_data)
#     vectores = vectorizer.toarray()
# #     dist = cosine_similarity(vectores, vectores)
#     dist = cosine_similarity(data11, data11)
#     print(dist.shape)
#     for k1, (doc_id1, title1, target_id1) in enumerate(docs1):
#         y_train1.append(target_id1)
#         groups_train1.append(new_group1)
#         words1 = set(title1.strip().split())
#         sum1 = 0
#         all_dist2 = []
#         all_dist4 = []
#         all_dist1 = []
#         for j in range(0, len(docs1)):
#             print(j)
#             if k1 == j:
#                 continue
#             doc_id_j, title_j, target_j = docs1[j]
#             words_j = set(title_j.strip().split())
#             y = len(words1.intersection(words_j))
#             y1 = Levenshtein.distance(title1,title_j)
#             y2 = dist[k1,j]
#             all_dist4.append(y2)
#             all_dist2.append(y1)
#             all_dist1.append(y)
#         for i in sorted(all_dist1, reverse=True)[0:15]:
#             sum1+=i
#         sum2 = 0
#         sum4 = 0
#         ss = 0
#         for i in sorted(all_dist2, reverse=False)[0:5]:
#             sum2+=i
#         for i in sorted(all_dist4, reverse=False)[0:15]:
#             sum4+=i
#         for i in sorted(all_dist2, reverse=False)[0:15]:
#             ss+=i
#         all_dist3=len(words1.intersection(dic))
#         X_train1.append(sorted(all_dist4, reverse=False)[0:15]+sorted(all_dist4, reverse=False)[15:2:25]+[all_dist3,sum4,sum1,ss,len(title1.strip().split())] + sorted(all_dist1, \
#                                 reverse=True)[0:5])
# X_train1 = np.array(X_train1)
# y_train1 = np.array(y_train1)
# groups_train1 = np.array(groups_train1)
# print (X_train1.shape, y_train1.shape, groups_train1.shape)

In [81]:
%%time
# вот этот кусок кода!!!!!!!!!!!!!!!!!!!

import numpy as np
import Levenshtein
import numpy as np
import gensim
import nltk as nl
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
y_train1 = []
X_train1 = []
groups_train1 = []
a = 0
b = 1000
for new_group1 in traingroups_titled:
    docs1 = traingroups_titled[new_group1]
    list_data = []
    ls = []
    aa = 0
    for k, (doc_id, title, target_id) in enumerate(docs1):
        title = title.strip().split()
        title1 = [u.lower() for u in title if not u.lower() in list_mest]
        list_data.append(' '.join(title1))
    vectorizer = TfidfVectorizer(max_features=10)
    X6 = vectorizer.fit_transform(list_data)
    dic = set(vectorizer.get_feature_names())
    print(dic)
    vectorizer = CountVectorizer(max_features=20).fit_transform(list_data)
    vectores = vectorizer.toarray()
    dist = cosine_similarity(X6, X6)
    for k1, (doc_id1, title1, target_id1) in enumerate(docs1):
        y_train1.append(target_id1)
        groups_train1.append(new_group1)
        words1 = set(title1.strip().split())
        sum1 = 0
        all_dist4 = []
        for j in range(0, len(docs1)):
            if k1 == j:
                continue
            doc_id_j, title_j, target_j = docs1[j]
            words_j = set(title_j.strip().split())
            y2 = dist[k1,j]
            all_dist4.append(y2)
        sum4 = 0
        for i in sorted(all_dist4, reverse=False)[0:15]:
            sum4+=i
        all_dist3=len(words1.intersection(dic))
        X_train1.append(sorted(all_dist4, reverse=False)[0:15]+[all_dist3,sum4,len(title1.strip().split())])
X_train1 = np.array(X_train1)
y_train1 = np.array(y_train1)
groups_train1 = np.array(groups_train1)
print (X_train1.shape, y_train1.shape, groups_train1.shape)

{'цена', 'ступица', 'сравнивать', 'купить', 'подшипник', 'нива', 'ремонт', 'замена', 'товар', 'ваз'}
{'здоровье', 'народный', 'лечить', 'форум', 'вопрос', 'ларингит', 'кашель', 'лечение', 'горло', 'дневник'}
{'врач', 'онлайн', 'история', 'цикл', 'день', 'сдавать', 'библиотека', 'вопрос', 'гормон', 'ттг'}
{'экономика', 'вкладывать', 'новость', 'фонд', 'инвестиция', 'финансы', 'деньги', 'пиф', 'инвестиционный', 'стоить'}
{'андреас', 'сан', 'гта', 'онлайн', 'скачать', 'мода', 'игра', 'карта', 'код', 'сервер'}
{'правильно', 'дизайн', 'должный', 'пошив', 'тюль', 'выбирать', 'фото', 'штора', 'пол', 'длина'}
{'почка', 'народный', 'пить', 'причина', 'много', 'день', 'полезный', 'вода', 'лечение', 'дневник'}
{'симптом', 'причина', 'аллергия', 'лечить', 'форум', 'насморк', 'ребенок', 'аллергический', 'лечение', 'ринит'}
{'симптом', 'народный', 'средство', 'лечить', 'долго', 'легкое', 'лечиться', 'туберкулез', 'время', 'лечение'}
{'санандрес', 'сделать', 'гта', 'прицел', 'скачать', 'игра', 'коман

{'форум', 'устанавливать', 'гта', 'дополнение', 'скачать', 'игра', 'омск', 'симс', 'играть', 'торрент'}
{'овощ', 'свежий', 'огурец', 'возраст', 'польза', 'помидор', 'ответ', 'ребенок', 'давать', 'год'}
{'пиво', 'здоровье', 'страница', 'форум', 'женский', 'онлайн', 'женщина', 'эстроген', 'гормон', 'лечение'}
{'страница', 'бокс', 'форум', 'гта', 'прошивка', 'скачать', 'игра', 'флешка', 'торрент', 'бесплатно'}
{'форум', 'радио', 'слушать', 'онлайн', 'блог', 'сайт', 'радиолюбитель', 'эфир', 'интернет', 'радиолюбительский'}
{'дача', 'огород', 'онлайн', 'желтеть', 'помидор', 'обсуждение', 'лист', 'российский', 'сервис', 'дневник'}
{'смотреть', 'онлайн', 'кинопоиск', 'звать', 'кино', 'хороший', 'фильм', 'трейлер', 'качество', 'бесплатно'}
{'мир', 'смотреть', 'рейтинг', 'онлайн', 'самый', 'фильм', 'хороший', 'сериал', 'крутой', 'год'}
{'правильно', 'лес', 'ориентирование', 'помощь', 'компас', 'местность', 'пользоваться', 'совет', 'ориентироваться', 'карта'}
{'санпин', 'организация', 'работа', 

In [116]:
scaler = StandardScaler()
scaler.fit(X_train3)
X_train3 = scaler.transform(X_train3)

In [112]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn import linear_model
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

max = []
where = 0
mmm = 0
for g in range(20):
    kf = KFold(n_splits=3)
    score = np.arange(60,dtype = float).reshape(3,-1)
    a = 0
    for train,test in kf.split(X_train3):

        X_tr = X_train3[train]
        y_tr = y_train1[train]
        X_t = X_train3[test]
        y_t = y_train1[test]
#         clf = LogisticRegression(random_state=0)
        clf = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        n_estimators = 1200
        learning_rate = 0.3
        max_depth = 6
#         clf2 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
#                         max_depth=max_depth, nthread=4)
#         clf2.fit(X_tr, y_tr)
#         pr = clf2.predict(X_t)
        clf.fit(X_tr, y_tr)
        my_clf_pred = clf.predict(X_t)
        
        
        for i in range(20,40):
    #         pred = (np.array(my_clf_pred>0.01*i,dtype=int)) 
            pred = my_clf_pred
            score[a][i-20] = f1_score(pred>0.01*(i),y_t)
        a+=1
    #     break
    if g==0:
        sc = score.mean(axis = 0)
    else:
        sc += score.mean(axis = 0)
    print(g)
    print(score.mean(axis = 0).max())
#     print(g,"score",sc.max())
#     max = sc.max()
#     print(np.where(sc==max)[0][0])
#     where +=np.where(sc==max)[0][0]
# #     print(sc)
# #     print(max,'max')
#     mmm+=sc.max()
# print('mean',mmm/20)
# print('mean',where/20)
print(sc.max()/20)
print(np.where(sc==sc.max())[0][0]+20)

/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


0
0.7147188007741626
1
0.7177463657772493
2
0.7126749144819651
3
0.7175941787833353
4
0.7166437295306762
5
0.707047468973454
6
0.713662411226495
7
0.70495567210651
8
0.7196675300192575
9
0.6914355236552997
10
0.7143617390325899
11
0.7074614247887535
12
0.7144888049602827
13
0.7167642417119312
14
0.707436273541754
15
0.7076802957368274
16
0.6970600309487168
17
0.717552526501216
18
0.71856366126929
19
0.7144097283556002
0.71025962335691
26


In [114]:
%%time
# вот этот кусок кода!!!!!!!!!!!!!!!!!!!

import numpy as np
import Levenshtein
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
y_train1 = []
X_train3 = []
groups_train1 = []
for new_group1 in traingroups_titled:
    docs1 = traingroups_titled[new_group1]
    list_data = []
    for k, (doc_id, title, target_id) in enumerate(docs1):
        title = title.strip().split()
        title1 = [u.lower() for u in title if not u.lower() in list_mest]
        list_data.append(' '.join(title1))
    vectorizer = TfidfVectorizer(max_features=20)
    X6 = vectorizer.fit_transform(list_data)
    dic = set(vectorizer.get_feature_names())
    print(dic)
    vectorizer = CountVectorizer(max_features=20).fit_transform(list_data)
    vectores = vectorizer.toarray()
    dist = cosine_similarity(X6, X6)
    for k1, (doc_id1, title1, target_id1) in enumerate(docs1):
        y_train1.append(target_id1)
        groups_train1.append(new_group1)
        words1 = set(title1.strip().split())
        sum1 = 0
        all_dist2 = []
        all_dist4 = []
        all_dist1 = []
        for j in range(0, len(docs1)):
            if k1 == j:
                continue
            doc_id_j, title_j, target_j = docs1[j]
            words_j = set(title_j.strip().split())
            y = len(words1.intersection(words_j))
            y1 = Levenshtein.distance(title1,title_j)
            y2 = dist[k1,j]
            all_dist4.append(y2)
            all_dist2.append(y1)
            all_dist1.append(y)
        for i in sorted(all_dist1, reverse=True)[0:15]:
            sum1+=i
        sum2 = 0
        sum4 = 0
        ss = 0
        for i in sorted(all_dist2, reverse=False)[0:5]:
            sum2+=i
        for i in sorted(all_dist4, reverse=False)[0:15]:
            sum4+=i
        for i in sorted(all_dist2, reverse=False)[0:15]:
            ss+=i
        all_dist3=len(words1.intersection(dic))
        X_train3.append(sorted(all_dist4, reverse=False)[0:15]+sorted(all_dist4, reverse=False)[15:2:25]+[all_dist3,sum4,sum1,ss,len(title1.strip().split())] + sorted(all_dist1, \
                                reverse=True)[0:5])
X_train3 = np.array(X_train3)
y_train1 = np.array(y_train1)
groups_train1 = np.array(groups_train1)
print (X_train3.shape, y_train1.shape, groups_train1.shape)

{'страница', 'потребительский', 'ступица', 'газ', 'товар', 'рука', 'подшипник', 'интернет', 'каталог', 'цена', 'автомобиль', 'магазин', 'нива', 'замена', 'ваз', 'передний', 'сравнивать', 'купить', 'ремонт', 'запчасть'}
{'здоровье', 'народный', 'домашний', 'кашель', 'лечение', 'целебный', 'тема', 'архив', 'вопрос', 'ребенок', 'условие', 'средство', 'причина', 'лечить', 'российский', 'ларингит', 'горло', 'беременность', 'дневник', 'форум'}
{'страница', 'щитовидный', 'история', 'цикл', 'лечение', 'анализ', 'архив', 'библиотека', 'вопрос', 'железа', 'онлайн', 'русский', 'день', 'сдавать', 'ттг', 'беременность', 'консультация', 'врач', 'гормон', 'терапия'}
{'глава', 'новость', 'предприятие', 'фонд', 'пиф', 'финансы', 'проект', 'стоить', 'выгодно', 'финансовый', 'инвестиция', 'реферат', 'бизнес', 'россия', 'экономика', 'вкладывать', 'форум', 'деньги', 'куда', 'инвестиционный'}
{'чит', 'машина', 'миссия', 'прохождение', 'код', 'андреас', 'скачать', 'мода', 'торрент', 'санандрес', 'сан', 'гта'

{'страница', 'новость', 'продаваться', 'город', 'архив', 'собака', 'казахстан', 'кызылорд', 'объявление', 'продажа', 'цена', 'информационный', 'область', 'животное', 'год', 'бизнес', 'форум', 'газета', 'купить', 'щенок'}
{'глава', 'устав', 'экстернат', 'образование', 'город', 'сош', 'муниципальный', 'общеобразовательный', 'школа', 'приказ', 'форма', 'российский', 'год', 'россия', 'учреждение', 'федеральный', 'закон', 'мбоу', 'новый', 'федерация'}
{'народный', 'похудение', 'домашний', 'похудеть', 'вылечивать', 'лечение', 'женский', 'отзыв', 'способ', 'условие', 'быстрый', 'средство', 'сайт', 'эффективный', 'медицина', 'диета', 'быстро', 'неделя', 'рецепт', 'помощь'}
{'страница', 'новость', 'обзор', 'андроид', 'смартфон', 'код', 'отзыв', 'чистый', 'тема', 'скачать', 'архив', 'видео', 'цена', 'гта', 'файл', 'книга', 'форум', 'купить', 'новый', 'мотоцикл'}
{'страница', 'работа', 'таблица', 'информатика', 'создание', 'документ', 'программа', 'один', 'тема', 'скачать', 'процессор', 'программ

{'устанавливать', 'город', 'омск', 'симс', 'скачать', 'мода', 'торрент', 'материал', 'выходить', 'гта', 'дополнение', 'онлайн', 'русский', 'игра', 'бесплатно', 'форум', 'сотовый', 'мобильный', 'видео', 'играть'}
{'овощ', 'свежий', 'огурец', 'возраст', 'помидор', 'яйцо', 'польза', 'ребенок', 'начинать', 'питание', 'детский', 'мес', 'соленый', 'давать', 'год', 'малыш', 'мама', 'месяц', 'ответ', 'рецепт'}
{'здоровье', 'страница', 'обсуждение', 'лечение', 'горький', 'пиво', 'женский', 'тема', 'эстроген', 'сервис', 'витамин', 'онлайн', 'дневник', 'медицина', 'форум', 'женщина', 'алкоголь', 'гормон', 'продукт', 'мужчина'}
{'чит', 'машина', 'страница', 'прошивка', 'код', 'бокс', 'скачать', 'торрент', 'смотреть', 'гта', 'икс', 'онлайн', 'сайт', 'игра', 'флешка', 'бесплатно', 'форум', 'самп', 'видео', 'играть'}
{'приемник', 'блог', 'любительский', 'программа', 'общение', 'радиосвязь', 'интернет', 'радиоприемник', 'автомобильный', 'слушать', 'онлайн', 'компьютер', 'сайт', 'радиолюбитель', 'радио

{'пошаговый', 'пенсионер', 'биометрический', 'загранпаспорт', 'госуслуга', 'документ', 'оформление', 'паспорт', 'интернет', 'украина', 'образец', 'получать', 'область', 'год', 'инструкция', 'россия', 'москва', 'оформлять', 'новый', 'портал'}
{'тело', 'тариф', 'кабинет', 'коннект', 'минута', 'личный', 'способ', 'узнавать', 'интернет', 'трафик', 'модем', 'оставаться', 'остаток', 'смарт', 'посмотреть', 'проверять', 'ответ', 'мтс', 'мобильный', 'мегафон'}
{'грамм', 'колхозница', 'худеть', 'дыня', 'фрукт', 'польза', 'полезный', 'свойство', 'вред', 'поправляться', 'калория', 'жлтый', 'противопоказание', 'сушеный', 'торпеда', 'арбуз', 'ответ', 'маленький', 'продукт', 'калорийность'}
{'здоровье', 'народный', 'домашний', 'вылечивать', 'рецепт', 'лечение', 'метод', 'шип', 'условие', 'симптом', 'средство', 'лечить', 'боль', 'фото', 'пяточный', 'эффективный', 'шпора', 'быстро', 'ответ', 'пятка'}
{'действие', 'гипертонический', 'блокатор', 'препарат', 'лечение', 'гипертония', 'список', 'отзыв', 'ле

In [60]:
params = product(
    (400, 600, 800, 1000, 1200), # n_estimators
    (0.01, 0.1, 0.2, 0.3, 0.4), # learning_rate
    (3, 6, 9), # max_depth
)
res_score = 0

for n_estimators, learning_rate, max_depth in list(params):
    clf = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
                        max_depth=max_depth, nthread=4)
#     clf = XGBClassifier(silent=False, 
#                       scale_pos_weight=1,
#                       learning_rate=0.01,  
#                       colsample_bytree = 0.4,
#                       subsample = 0.8,
#                       objective='binary:logistic', 
#                       n_estimators=1000, 
#                       reg_alpha = 0.3,
#                       max_depth=9, 
#                       gamma=10)
    res = cross_validate(clf, X_train1, y_train1, cv=3, scoring='f1')
    score = res['test_score'].mean()
    print(score)
    print(clf)
    if score > res_score:
        res_score = score
        res_model = clf
        
res_model, res_score

0.6712117466091195
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=None, max_depth=3,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=400, n_jobs=None, nthread=4, num_parallel_tree=None,
              objective='binary:logistic', random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None)
0.6793432535358527
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=None, max_depth=6,
   

KeyboardInterrupt: 

In [61]:
import numpy as np
y_train = []
X_train = []
groups_train = []
for new_group in traingroups_titled:
    docs = traingroups_titled[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        y_train.append(target_id)
        groups_train.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        sum1 = 0
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            y = len(words.intersection(words_j))
            all_dist.append(y)
            sum1+=y
        X_train.append(sorted(all_dist, reverse=True)[0:15] +[sum1,len(title.strip().split())]   )
X_train = np.array(X_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print (X_train.shape, y_train.shape, groups_train.shape)

(11690, 17) (11690,) (11690,)


In [62]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

In [63]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn import linear_model
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

max = []
where = 0
mmm = 0
for g in range(20):
    kf = KFold(n_splits=3)
    score = np.arange(60,dtype = float).reshape(3,-1)
    a = 0
    for train,test in kf.split(X_train):

        X_tr = X_train[train]
        y_tr = y_train1[train]
        X_t = X_train[test]
        y_t = y_train1[test]
#         clf = LogisticRegression(random_state=0)
        clf = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        n_estimators = 1200
        learning_rate = 0.3
        max_depth = 6
#         clf2 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
#                         max_depth=max_depth, nthread=4)
#         clf2.fit(X_tr, y_tr)
#         pr = clf2.predict(X_t)
        clf.fit(X_tr, y_tr)
        my_clf_pred = clf.predict(X_t)
        
        
        for i in range(20,40):
    #         pred = (np.array(my_clf_pred>0.01*i,dtype=int)) 
            pred = my_clf_pred
            score[a][i-20] = f1_score(pred>0.01*(i),y_t)
        a+=1
    #     break
    if g==0:
        sc = score.mean(axis = 0)
    else:
        sc += score.mean(axis = 0)
    print(g)
    print(score.mean(axis = 0).max())
#     print(g,"score",sc.max())
#     max = sc.max()
#     print(np.where(sc==max)[0][0])
#     where +=np.where(sc==max)[0][0]
# #     print(sc)
# #     print(max,'max')
#     mmm+=sc.max()
# print('mean',mmm/20)
# print('mean',where/20)
print(sc.max()/20)
print(np.where(sc==sc.max())[0][0]+20)

0
0.7147812679244424


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


1
0.7171309922038865
2
0.7095076987839075
3
0.7127048537417462
4
0.7153147274069158
5
0.7145828374038103
6
0.7135820387582212
7
0.7160692404940773
8
0.7163551352065763
9
0.6439434488153455
10
0.7124890479198672
11
0.7163052204370314
12
0.7146874121183462
13
0.7093930780502679
14
0.7159533475238414
15
0.7160155082917768
16
0.7155464373044902
17
0.7177346668411569
18
0.708812399506724
19
0.7135731563590643
0.710096349364177
26


In [52]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
max = []
where = 0
mmm = 0
for g in range(20):
    kf = KFold(n_splits=3)
    score = np.arange(60,dtype = float).reshape(3,-1)
    a = 0
    for train,test in kf.split(X_train):

        X_tr = X_train[train]
        y_tr = y_train1[train]
        X_t = X_train[test]
        y_t = y_train1[test]
        X_tr1 = X_train1[train]
        X_t1 = X_train1[test]
#         clf = LogisticRegression(random_state=0)
        clf = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        clf.fit(X_tr, y_tr)
        my_clf_pred = clf.predict(X_t)
        res1 = (my_clf_pred>0.01*(26)).reshape(-1,1)
        
        
        
        
        n_estimators = 1200
        learning_rate = 0.001
        max_depth = 6
        clf2 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
                        max_depth=max_depth, nthread=4)
        clf2.fit(X_tr, y_tr)
        pr = clf2.predict(X_t).reshape(-1,1)
        clf1 = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        clf1.fit(X_tr1, y_tr)
        my_clf_pred1 = clf1.predict(X_t1)
        res2 = (my_clf_pred1>0.01*(28)).reshape(-1,1)
        clf3 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
                        max_depth=max_depth, nthread=4)
        clf3.fit(X_tr1, y_tr)
        pr1 = clf3.predict(X_t1).reshape(-1,1)
        
        X_res = np.concatenate((res1,pr,res2,pr1), axis=1)
        y_res =  (np.mean(X_res, axis = 1))>0.5
        print(X_res.shape)
        print(y_res.shape)
        print(y_t.shape)
        for i in range(20,40):
    #         pred = (np.array(my_clf_pred>0.01*i,dtype=int)) 
            pred = my_clf_pred
            score[a][i-20] = f1_score(y_res,y_t)
        a+=1
    #     break
    if g==0:
        sc = score.mean(axis = 0)
    else:
        sc += score.mean(axis = 0)
    print(g)
    print(score.mean(axis = 0).max())
#     print(g,"score",sc.max())
#     max = sc.max()
#     print(np.where(sc==max)[0][0])
#     where +=np.where(sc==max)[0][0]
# #     print(sc)
# #     print(max,'max')
#     mmm+=sc.max()
# print('mean',mmm/20)
# print('mean',where/20)
print(sc.max()/20)
print(np.where(sc==sc.max())[0][0]+20)

/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


(3897, 4)
(3897,)
(3897,)


KeyboardInterrupt: 

In [64]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
max = []
where = 0
mmm = 0
www = np.zeros(4)
for g in range(5):
    kf = KFold(n_splits=3)
    score = np.arange(60,dtype = float).reshape(3,-1)
    a = 0
    for train,test in kf.split(X_train):

        X_tr = X_train[train]
        y_tr = y_train1[train]
        X_t = X_train[test]
        y_t = y_train1[test]
        X_tr1 = X_train1[train]
        X_t1 = X_train1[test]
#         clf = LogisticRegression(random_state=0)
        clf = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        clf.fit(X_tr, y_tr)
        my_clf_pred = clf.predict(X_t)
        res1 = (my_clf_pred>0.01*(26)).reshape(-1,1)
        
        
        
        
        n_estimators = 1200
        learning_rate = 0.001
        max_depth = 6
        clf2 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
                        max_depth=max_depth, nthread=4)
        clf2.fit(X_tr, y_tr)
        pr = clf2.predict(X_t).reshape(-1,1)
        clf1 = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        clf1.fit(X_tr1, y_tr)
        my_clf_pred1 = clf1.predict(X_t1)
        res2 = (my_clf_pred1>0.01*(29)).reshape(-1,1)
        clf3 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
                        max_depth=max_depth, nthread=4)
        clf3.fit(X_tr1, y_tr)
        pr1 = clf3.predict(X_t1).reshape(-1,1)
        
        X_res = np.concatenate((res1,pr,res2,pr1), axis=1)
#         y_res =  (np.mean(X_res, axis = 1))>0.5
        print(X_res.shape)
        print(y_res.shape)
        print(y_t.shape)
        clf5 = MySGDClassifier(batch_generator, model_type="lin_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        reg = LinearRegression().fit(X_res, y_t)
        www+=reg.coef_
        print(www/(g*3+3))
#     print(g,"score",sc.max())
#     max = sc.max()
#     print(np.where(sc==max)[0][0])
#     where +=np.where(sc==max)[0][0]
# #     print(sc)
# #     print(max,'max')
#     mmm+=sc.max()
# print('mean',mmm/20)
# print('mean',where/20)
print(www/10)

(3897, 4)
(3897, 1)
(3897,)
[0.07167395 0.03481372 0.06852429 0.05661568]


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


(3897, 4)
(3897, 1)
(3897,)
[0.14806138 0.10264635 0.10317016 0.12068825]


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


(3896, 4)
(3897, 1)
(3896,)
[0.1749663  0.17144779 0.16566921 0.21521665]


...

(3896, 4)
(3897, 1)
(3896,)
[0.19565932 0.16159799 0.16386877 0.20445488]
(3897, 4)
(3897, 1)
(3897,)
[0.16777848 0.13660315 0.1463222  0.17652838]


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


(3897, 4)
(3897, 1)
(3897,)
[0.1851152  0.15009925 0.15124494 0.18943829]


/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


(3896, 4)
(3897, 1)
(3896,)
[0.19721109 0.16177852 0.15989065 0.20706869]
[0.29581664 0.24266778 0.23983597 0.31060303]


In [38]:
www/30

array([0.22154872, 0.14664111, 0.14601091, 0.20563443])

In [65]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
max = []
where = 0
mmm = 0
for g in range(20):
    kf = KFold(n_splits=3)
    score = np.arange(300,dtype = float).reshape(3,-1)
    a = 0
    for train,test in kf.split(X_train):

        X_tr = X_train[train]
        y_tr = y_train1[train]
        X_t = X_train[test]
        y_t = y_train1[test]
        X_tr1 = X_train1[train]
        X_t1 = X_train1[test]
#         clf = LogisticRegression(random_state=0)
        clf = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        clf.fit(X_tr, y_tr)
        my_clf_pred = clf.predict(X_t)
        res1 = (my_clf_pred>0.01*(26)).reshape(-1,1)
        
        
        
        
        n_estimators = 1200
        learning_rate = 0.001
        max_depth = 6
        clf2 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
                        max_depth=max_depth, nthread=4)
        clf2.fit(X_tr, y_tr)
        pr = clf2.predict(X_t).reshape(-1,1)
        clf1 = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                      alpha = 0.01, C=50,batch_size = 100)
        clf1.fit(X_tr1, y_tr)
        my_clf_pred1 = clf1.predict(X_t1)
        res2 = (my_clf_pred1>0.01*(29)).reshape(-1,1)
        clf3 = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
                        max_depth=max_depth, nthread=4)
        clf3.fit(X_tr1, y_tr)
        pr1 = clf3.predict(X_t1).reshape(-1,1)
        
        X_res = np.concatenate((res1,pr,res2,pr1), axis=1)
        y_res =  res1*0.29581664+pr* 0.24266778+res2* 0.23983597+pr1* 0.20563443
#         print(X_res.shape)
#         print(y_res.shape)
#         print(y_t.shape)
        for i in range(0,100):
    #         pred = (np.array(my_clf_pred>0.01*i,dtype=int)) 
            pred = my_clf_pred
            score[a][i] = f1_score(y_res>0.01,y_t)
        a+=1
    #     break
    if g==0:
        sc = score.mean(axis = 0)
    else:
        sc += score.mean(axis = 0)
    print(g)
    print(score.mean(axis = 0).max())
#     print(g,"score",sc.max())
#     max = sc.max()
#     print(np.where(sc==max)[0][0])
#     where +=np.where(sc==max)[0][0]
# #     print(sc)
# #     print(max,'max')
#     mmm+=sc.max()
# print('mean',mmm/20)
# print('mean',where/20)
print(sc.max()/20)
print(np.where(sc==sc.max())[0][0])

/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


0
0.7188433406582219
1
0.7059350432772873
2
0.7185650851114782
3
0.7166568719300007
4
0.7217715102771858
5
0.7238488313578063
6
0.7177604271392327
7
0.7188453819155849


KeyboardInterrupt: 

In [87]:
test_data1 = pd.read_csv('test_groups.csv')
test_groups_titled1 = {}
for i in range(len(test_data1)):
    new_doc1 = test_data1.iloc[i]
    doc_group1 = new_doc1['group_id']
    doc_id1 = new_doc1['doc_id']
    title1 = doc_to_t[doc_id1]
    
#     title1 = [u.lower() for u in title1 if not u.lower() in list_mest]
#     title1 = [morph.parse(o)[0].normal_form for o in title1]
    if doc_group1 not in test_groups_titled1:
        test_groups_titled1[doc_group1] = []
    test_groups_titled1[doc_group1].append((doc_id1, title1))

In [115]:
%%time
X_test1 = []
groups_test1 = []
p1 = 0
for new_group in test_groups_titled1:
    p1+=1
    print(p1)
    docs1 = test_groups_titled1[new_group]
    list_data = []
    for k, (doc_id, title) in enumerate(docs1):
        title = title.strip().split()
        title1 = [u.lower() for u in title if not u.lower() in list_mest]
        list_data.append(' '.join(title1))
    vectorizer = TfidfVectorizer(max_features=20)
    X6 = vectorizer.fit_transform(list_data)
    dic = set(vectorizer.get_feature_names())
    print(dic)
    vectorizer = CountVectorizer(max_features=20).fit_transform(list_data)
    vectores = vectorizer.toarray()
    dist = cosine_similarity(X6, X6)
    for k, (doc_id, title) in enumerate(docs1):
        groups_test1.append(new_group)
        all_dist4 = []
        all_dist2 = []
        words = set(title.strip().split())
        sum1 = 0
#         dist = np.zeros((len(docs1), len(docs1)))
        all_dist1 = []
        n = 0
        for j in range(0, len(docs1)):
            if k == j:
                continue
            n+=1
            doc_id_j, title_j= docs1[j]
            words_j = set(title_j.strip().split())
            y = len(words.intersection(words_j))
            y1 = Levenshtein.distance(title,title_j)
            all_dist2.append(y1)
            y2 = dist[k,j]
#             print(y2)
            all_dist4.append(y2)
            all_dist1.append(y)
#             sum1+=y
        for i in sorted(all_dist1, reverse=True)[0:15]:
            sum1+=i
        sum2 = 0
        for i in sorted(all_dist2, reverse=False)[0:5]:
            sum2+=i
        sum4 = 0
        for i in sorted(all_dist4, reverse=False)[0:15]:
            sum4+=i
        ss = 0
        for i in sorted(all_dist2, reverse=False)[0:15]:
            ss+=i
        all_dist3=len(words.intersection(dic))
        X_test1.append(sorted(all_dist4, reverse=False)[0:15] +sorted(all_dist4, reverse=False)[15:2:25]+[all_dist3, sum4,sum1,ss,len(title.strip().split())] +sorted(all_dist1, reverse=True)[0:5]  )
#         print(sum2)
X_test1 = np.array(X_test1)
groups_test1 = np.array(groups_test1)
print (X_test1.shape, groups_test1.shape)

1
{'страница', 'плагин', 'ксс', 'блог', 'админ', 'прописывать', 'скачать', 'интернет', 'создавать', 'гта', 'админка', 'сайт', 'игра', 'давать', 'форум', 'самп', 'статья', 'видео', 'помощь', 'сервер'}
2
{'работа', 'международный', 'центр', 'контакт', 'телефон', 'отзыв', 'мякинино', 'вакансия', 'добираться', 'крокус', 'метро', 'магазин', 'адрес', 'россия', 'строгино', 'сити', 'москва', 'группа', 'компания', 'торговый'}
3
{'мир', 'погода', 'балашовский', 'точный', 'новость', 'самый', 'тема', 'изюм', 'саратовский', 'русский', 'день', 'область', 'почасовой', 'дневник', 'год', 'россия', 'завтра', 'район', 'балашов', 'прогноз'}
4
{'мир', 'страница', 'новость', 'сода', 'обсуждение', 'лечение', 'магия', 'тема', 'скачать', 'сервис', 'ожог', 'онлайн', 'российский', 'книга', 'дневник', 'бесплатно', 'растение', 'форум', 'ядовитый', 'борщевик'}
5
{'вконтакте', 'домашний', 'обсуждение', 'лечение', 'зима', 'черный', 'заголовок', 'сервис', 'смородина', 'онлайн', 'фото', 'российский', 'магазин', 'дневни

38
{'вконтакте', 'страница', 'новость', 'правило', 'акпп', 'эксплуатация', 'клуб', 'архив', 'масло', 'владелец', 'часть', 'жидкость', 'автомобиль', 'википедия', 'техника', 'просмотр', 'замена', 'книга', 'форум', 'ремонт'}
39
{'здоровье', 'профилактика', 'герпес', 'вылечивать', 'кашель', 'лечение', 'гастрит', 'заболевание', 'ребенок', 'язык', 'условие', 'симптом', 'причина', 'медицинский', 'лечить', 'онлайн', 'стерлитамак', 'стоматит', 'взрослый', 'портал'}
40
{'работа', 'выбирать', 'линза', 'обсуждение', 'салон', 'лечение', 'зрение', 'очко', 'отзыв', 'солнцезащитный', 'контактный', 'бифокальный', 'интернет', 'медицинский', 'очки', 'цена', 'москва', 'оптика', 'глаз', 'портал'}
41
{'страница', 'совет', 'препарат', 'лечение', 'миопатия', 'отзыв', 'помогать', 'скачать', 'заболевание', 'ребенок', 'средство', 'применение', 'онлайн', 'педиатрия', 'показание', 'инструкция', 'врач', 'форум', 'дибазол', 'помощь'}
42
{'научно', 'образовательный', 'страница', 'новость', 'образование', 'комиссия', 

78
{'родосский', 'история', 'искусство', 'тема', 'скачать', 'архив', 'библиотека', 'часть', 'культура', 'онлайн', 'греция', 'игра', 'книга', 'дневник', 'читать', 'бесплатно', 'цивилизация', 'форум', 'древний', 'колосс'}
79
{'блюдо', 'стерлядь', 'пошаговый', 'целиком', 'повар', 'запекать', 'приготавливать', 'фольга', 'вкусный', 'кулинарный', 'целый', 'фото', 'духовка', 'приготовление', 'готовить', 'стерлядка', 'ответ', 'рецепт', 'осетр', 'рыба'}
80
{'глаголать', 'мир', 'страница', 'юрист', 'дело', 'международный', 'тема', 'архив', 'вопрос', 'право', 'авторский', 'онлайн', 'просмотр', 'уголовный', 'дневник', 'гражданин', 'правовой', 'регистрация', 'форум', 'зарегистрировать'}
81
{'чит', 'вконтакте', 'блог', 'симс', 'обзор', 'код', 'чита', 'скачать', 'мода', 'навык', 'грудь', 'дополнение', 'игра', 'бесплатно', 'предмет', 'форум', 'размер', 'увеличивать', 'деньги', 'секрет'}
82
{'еда', 'делать', 'домашний', 'блюдо', 'совет', 'пища', 'гречка', 'макароны', 'соль', 'фото', 'спасать', 'дневник

116
{'прибыль', 'расход', 'бухгалтерия', 'пример', 'работа', 'новость', 'учет', 'госпошлина', 'проводка', 'продажа', 'российский', 'доход', 'система', 'налоговый', 'год', 'россия', 'экономика', 'форум', 'дипломный', 'налог'}
117
{'стенка', 'страница', 'доставка', 'совет', 'обзор', 'почему', 'задний', 'отзыв', 'намерзать', 'интернет', 'цена', 'техника', 'магазин', 'санкт', 'лед', 'инструкция', 'весь', 'характеристика', 'купить', 'холодильник'}
118
{'страница', 'бухгалтерия', 'работа', 'труд', 'учет', 'сотрудник', 'оплата', 'архив', 'вопрос', 'бухгалтер', 'производственный', 'оклад', 'работник', 'год', 'зарплата', 'форум', 'повышение', 'ответ', 'организация', 'ооо'}
119
{'оператор', 'дозваниваться', 'подмосковье', 'тариф', 'клиент', 'связь', 'телефон', 'интернет', 'номер', 'услуга', 'область', 'бесплатно', 'москва', 'телевидение', 'сотовый', 'ответ', 'частный', 'позвонить', 'мтс', 'мобильный'}
120
{'длинный', 'пиджак', 'жакет', 'выбирать', 'рукав', 'модель', 'совет', 'разговор', 'стильны

154
{'обсуждение', 'снимать', 'определять', 'скачать', 'узнавать', 'библиотека', 'сервис', 'способ', 'молитва', 'порча', 'онлайн', 'российский', 'вода', 'книга', 'дневник', 'бесплатно', 'сглазить', 'форум', 'заговор', 'сглаз'}
155
{'прошивка', 'обзор', 'андроид', 'смартфон', 'телефон', 'версия', 'отзыв', 'интернет', 'планшет', 'цена', 'онлайн', 'иксион', 'магазин', 'дешево', 'купить', 'мобигура', 'устройство', 'мобильный', 'видео', 'дексп'}
156
{'плагин', 'устанавливать', 'страйк', 'модель', 'игровой', 'тренировка', 'скачать', 'мода', 'бот', 'контры', 'хороший', 'торрент', 'патч', 'игра', 'ссылка', 'карта', 'бесплатно', 'мониторинг', 'загрузка', 'сервер'}
157
{'предприятие', 'юридический', 'лицо', 'директор', 'суд', 'создание', 'учредитель', 'нужный', 'договор', 'вопрос', 'право', 'общество', 'год', 'правовой', 'консультация', 'регистрация', 'форум', 'учредительный', 'государственный', 'ооо'}
158
{'здоровье', 'страница', 'лечение', 'женский', 'диагностика', 'ребенок', 'мочеиспускание',

In [117]:
scaler = StandardScaler()
scaler.fit(X_test1)
X_test1 = scaler.transform(X_test1)

In [118]:
# score на kaggle 0,693
# модель с косинусной схожестью(ансамбль моделей и word2vec пока не делала), использовала my stem вместо pymorphy
clf1 = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=6,
                  alpha = 0.01, C=50,batch_size = 100)
clf1.fit(X_train3, y_train1)
my_clf_pred1 = clf1.predict(X_test1)
# clf = MySGDClassifier(batch_generator, model_type="log_reg", max_epoch=10,
#                   alpha = 0.01, C=1,batch_size = 100)
# clf.fit(X_train, y_train)
# my_clf_pred = clf.predict(X_test)
pred = my_clf_pred1 
print(np.array(pred>0.29,dtype = int).sum())
test_data1["target"] = pd.Series(np.array(pred>0.29,dtype = int))
with open("pred_12.csv", "w") as f:
    f.write(test_data1.to_csv(columns=("pair_id", "target"), index=False))

/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/home/nastya/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


5193


In [119]:
y_train1.sum()/y_train1.shape[0]

0.28751069289991443

In [120]:
(pred>0.29).sum()/pred.shape[0]

0.3123233295242678